In [1]:
import sys
import os

# Get the absolute path of the project root
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Add it to sys.path if not already added
if project_root not in sys.path:
    sys.path.append(project_root)

In [2]:
from src.functions.data_generation import DataGeneration
from sklearn.model_selection import train_test_split

data = DataGeneration.generate_data(
    n_samples=10000,
    n_features=20,
    n_informative=4,
    relationship='friedman2',
    noise=0.1,
    random_state=58495
)

# Draw bootstrap sample
bootstrap_data = data.sample(n=1000, replace=True, random_state=89321)
bootstrap_data = DataGeneration.add_epsilon(bootstrap_data)

display(bootstrap_data)

X = bootstrap_data.drop(columns=["Target"])
y = bootstrap_data["Target"]

# Split data
X_proper, X_test, y_proper, y_test = train_test_split(X, y, test_size=0.5)

        
def to_numpy_safe(x):
    return x.to_numpy() if hasattr(x, "to_numpy") else x

# Convert to numpy arrays if needed
X_proper = to_numpy_safe(X_proper)
y_proper = to_numpy_safe(y_proper)
X_test   = to_numpy_safe(X_test)
y_test   = to_numpy_safe(y_test)

,Feature_0,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,Feature_8,Feature_9,...,Feature_11,Feature_12,Feature_13,Feature_14,Feature_15,Feature_16,Feature_17,Feature_18,Feature_19,Target
1907,66.358060,1042.595014,0.416862,9.759544,0.141956,0.536689,0.920227,0.811599,0.724678,0.654342,...,0.318806,0.012214,0.992264,0.060583,0.149041,0.897780,0.093677,0.008575,0.795700,0.249900
1836,12.164998,574.252129,0.208135,1.643181,0.211437,0.472563,0.284444,0.489938,0.338878,0.940321,...,0.621251,0.889889,0.810347,0.324495,0.183444,0.045404,0.010770,0.172390,0.247675,0.067587
9843,66.008198,1277.479355,0.485108,2.093126,0.341579,0.371495,0.517523,0.913813,0.998465,0.122765,...,0.318679,0.266682,0.678660,0.558199,0.794302,0.093753,0.418159,0.922168,0.671439,0.354656
5340,84.886851,1013.863640,0.148065,2.256299,0.560992,0.094883,0.049354,0.590603,0.418445,0.104658,...,0.395214,0.525198,0.922072,0.549223,0.899359,0.927153,0.257011,0.119961,0.834851,0.097398
7901,41.417166,809.436576,0.600055,7.808032,0.100598,0.009912,0.056442,0.446325,0.136095,0.710399,...,0.317341,0.732263,0.577633,0.770164,0.278270,0.382352,0.532663,0.954255,0.661659,0.277290
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3917,78.003049,1103.288571,0.591527,8.566431,0.899770,0.187217,0.881093,0.080200,0.023833,0.155888,...,0.832718,0.995019,0.723433,0.644408,0.189370,0.228061,0.065652,0.561389,0.928129,0.374134
2854,62.072633,469.051915,0.825910,4.406076,0.104684,0.936994,0.578636,0.156674,0.943286,0.285292,...,0.320164,0.385482,0.054800,0.504555,0.743523,0.023226,0.923234,0.992276,0.762523,0.223018
1080,57.629681,756.533935,0.563832,2.092401,0.316023,0.771101,0.614691,0.225276,0.076348,0.073821,...,0.983625,0.604988,0.170375,0.151545,0.808739,0.291994,0.748015,0.371071,0.689662,0.244684
8401,4.557764,1659.168426,0.651357,7.935800,0.370019,0.100569,0.876258,0.410138,0.025383,0.520405,...,0.893629,0.260990,0.225964,0.690517,0.691473,0.896692,0.426713,0.760585,0.173566,0.615900


In [5]:
from online_cp.CPS import NearestNeighboursPredictionMachine
from online_cp.CPS import RidgePredictionMachine, KernelRidgePredictionMachine
import numpy as np
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C

# cps = NearestNeighboursPredictionMachine(k=5)
# cps = RidgePredictionMachine(autotune=True)
kernel = C(1.0, constant_value_bounds=(1e-7, 1e7)) * RBF(length_scale=1.0, length_scale_bounds=(1e-7, 1e7))
cps = KernelRidgePredictionMachine(a=1000, kernel=kernel, autotune=True)

cps.learn_initial_training_set(X_proper, y_proper)


# x = [7.34768661e+01, 
#      3.07355747e+02, 
#      9.98483471e-01, 
#      9.49902854e+00,
#      3.62115400e-01, 
#      5.48508689e-01, 
#      8.40112768e-01, 
#      3.42234629e-01,
#      3.44227224e-01, 
#      7.71883283e-01, 
#      2.42761017e-01, 
#      3.44293874e-01,
#      5.20936857e-01, 
#      5.20083931e-01, 
#      8.81639709e-01, 
#      3.41171486e-01,
#      1.60554105e-01, 
#      5.98680164e-01, 
#      7.88717442e-01, 
#      7.44306979e-02]

for i, (x, y) in enumerate(zip(X_test, y_test)):
    #try:

    cpd, precomputed = cps.predict_cpd(x=x, return_update=True)
    
    # except Exception as e:
    #     print(i)
    #     print()
    #     print("Exception occurred for x =", x)
    #     print()
    #     print("Error:", e)
    

[      -inf 0.01129553 0.01351978 0.01407863 0.01537661 0.01786703
 0.02040642 0.02095001 0.02118395 0.02191625 0.02258787 0.02397845
 0.02456063 0.02477529 0.02614099 0.02625845 0.02630248 0.02669447
 0.02772308 0.02782916 0.02834256 0.0294153  0.02948953 0.02975697
 0.03095973 0.03227197 0.03243203 0.03387395 0.03426111 0.03503121
 0.03509264 0.03606072 0.03609711 0.03644729 0.03934665 0.0397139
 0.04048951 0.04089565 0.04170284 0.04189933 0.04323159 0.04365076
 0.04394537 0.0449286  0.0454379  0.04641278 0.04690462 0.04743185
 0.0480288  0.04808046 0.04932559 0.05016809 0.05080013 0.05149527
 0.05221168 0.05267848 0.05287897 0.05363571 0.05412744 0.05416777
 0.05460383 0.05481483 0.05505665 0.0557367  0.05656483 0.05670169
 0.05673442 0.05682456 0.05813342 0.05827931 0.05883714 0.05899783
 0.05905008 0.05969264 0.0602577  0.06026075 0.06076662 0.06091295
 0.06107223 0.06178125 0.0618977  0.06235191 0.06235191 0.06254405
 0.06471581 0.06479476 0.0648137  0.06510865 0.06515376 0.06595